# transform module

Available methods:

- check_points_within_ellipse: checks if points are grouped within specified ellipse for directional semivariograms,
- get_centroids: gets position x and y of polygon centroid,
- prepare_kriging_data: prepares data for kriging - array of point position, value and distance to an unknown point,
- prepare_poisson_kriging_data: prepares data for centroid based Poisson Kriging,
- prepare_ata_data: prepares data for Area to Area Poisson Kriging,
- prepare_atp_data: prepares data for Area to Point Poisson Kriging,
- select_values_in_range: selects set of values which are greater than (lag - step size) and lesser than (lag + step size),
- set_areal_weights: prepares array for weighted semivariance calculation.

***

## ```check_points_within_ellipse()```

```python
pyinterpolate.transform.check_points_within_ellipse(
    origin_point,
    other_points,
    step_size,
    last_step_size,
    angle,
    tolerance
)
```

Function checks which points from other points are within point range described as an ellipse with
center in an `origin_point`, semi-major axis of length `step_size` and semi-minor axis of length
`step_size * tolerance` and angle of semi-major axis calculated as `angle` of direction from
NS (0 radian angle) axis of a dataset.

INPUT:

- **origin_point**: (*numpy array*) single point coordinates,
- **other_points**: (*numpy array*),
- **step_size**: (*float*) current distance between lags within each points are included in the calculations,
- **last_step_size**: (*float*) last distance between lags within each points are included in the calculations,
- **angle**: (*float*) direction of semivariogram, in radians. Angle is rotated by PI/2 rad.
- **tolerance**: (*float*) value in range 0-1 normalized to `[0 : 0.5]` to select tolerance of semivariogram. If
tolerance is 0 then points must be placed at a single line with beginning in the origin of coordinate
system and angle given by y axis and direction parameter. If tolerance is greater than 0 then semivariance
is estimated from elliptical area with major axis with the same direction as the line for 0 tolerance
and minor axis of a size:

$$(tolerance * step\_size)$$

and major axis (pointed in NS direction):

$$((1 - tolerance) * step\_size)$$

and baseline point at a center of ellipse. `tolerance == 1` creates omnidirectional semivariogram.

#### ROTATED ELLIPSE EQUATION:

$$part_a = [cos(A) * (x - h) + sin(A) * (y - k)]^2$$

$$part_b = [sin(A) * (x - h) + cos(A) * (y - k)]^2$$

and if:

$$\frac{part_a}{r_x^2} + \frac{part_b}{r_y^2} <= 1$$

then point is inside ellipse.

OUTPUT:

(*numpy array*) boolean array of **points within ellipse with center in origin point**.

***

## ```get_centroids()```

```python
pyinterpolate.transform.get_centroids(polygon)
```

Function gets position x and y of polygon centroid.


INPUT:

- **polygon**: (_shapely Polygon_).


OUTPUT:

- (*tuple*) centroid position (x, y) for a given area.

***

## ```prepare_kriging_data()```

```python
pyinterpolate.transform.prepare_kriging_data(
    unknown_position,
    data_array,
    number_of_neighbours=10)
```

Function prepares data for kriging - array of point position, value and distance to an unknown point.


INPUT:

- **unknown_position**: (_numpy array_) position of unknown value,
- **data_array**: (_numpy array_) known positions and their values,
- **number_of_neighbours**: (_int_) number of the closest locations to the unknown position.


OUTPUT:

- (```numpy array```) dataset with position, value and distance to the unknown point:

```python
[
    [x, y, value, distance to unknown position],
    [...]
]
```

***

## ```prepare_poisson_kriging_data()```

```python
pyinterpolate.transform.prepare_poisson_kriging_data(
    unknown_area,
    points_within_unknown_area,
    known_areas,
    points_within_known_areas,
    number_of_neighbours,
    max_search_radius,
    weighted=False)
```

Function prepares data for centroid based Poisson Kriging.


INPUT:

- **unknown_area**: (_numpy array_) unknown area in the form: 

```python
[area_id, polygon, centroid x, centroid y]
```
- **points_within_unknown_area**: (_numpy array_) points and their values within the given area:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```
- **known_areas**: (_numpy array_) known areas in the form:

```python
[area_id, polygon, centroid x, centroid y, aggregated value]
```
- **points_within_known_areas**: (_numpy array_) points and their values within the given area:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```
- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than ```number_of_neighbours``` parameter then additional neighbours are included up to the minimum number of neighbors),
- **weighted**: (_bool_) distances weighted by process or point measurements within area (```True```) or not (```False```).


OUTPUT:

- (```numpy array```) distances from known locations to the unknown location:

```python
[
    id_known,
    coordinate x,
    coordinate y,
    value,
    distance to unknown,
    aggregated points values within an area
]
```

***

## ```prepare_ata_data()```

```python
pyinterpolate.transform.prepare_ata_data(
    points_within_unknown_area,
    known_areas,
    points_within_known_areas,
    number_of_neighbours,
    max_search_radius)
```

Function prepares data for Area to Area Poisson Kriging.


INPUT:

- **points_within_unknown_area**: (_numpy array_) points and their values within the unknown area:

```python
[
    area_id,
    [point_position_x, point_position_y, value of point]
]
```
- **known_areas**: (_numpy array_) known areas in the form:

```python
[area_id, polygon, centroid x, centroid y, aggregated value]
```
- **points_within_known_areas**: (_numpy array_) points and their values within the given area:

```python
[
    [area_id,
    [point_position_x, point_position_y, value of point]]
]
```
- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than ```number_of_neighbours``` parameter then additional neighbours are included up to the minimum number of neighbors).


OUTPUT:

- (```numpy array```) distances from known locations to the unknown location:

```python
[
    id_known,
    areal value - count,
    [known_point_1 value, unknown_point_1 value, distance_1],
    total point value
]
```

***

## ```prepare_atp_data()```

```python
pyinterpolate.transform.prepare_atp_data(
    points_within_unknown_area,
    known_areas,
    points_within_known_areas,
    number_of_neighbours,
    max_search_radius)
```

Function prepares data for Area to Point Poisson Kriging.


INPUT:

- **points_within_unknown_area**: (_numpy array_) points and their values within the unknown area:

```python
[
    area_id,
    [point_position_x, point_position_y, value of point]
]
```
- **known_areas**: (_numpy array_) known areas in the form:

```python
[area_id, polygon, centroid x, centroid y, aggregated value]
```
- **points_within_known_areas**: (_numpy array_) points and their values within the given area:

```python
[
    [area_id,
    [point_position_x, point_position_y, value of point]]
]
```
- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than ```number_of_neighbours``` parameter then additional neighbours are included up to the minimum number of neighbors).


OUTPUT:

- (```numpy array```) distances from known locations to the unknown location:

```python
[
    id_known,
    areal value - count,
    [
        unknown point value, 
        [known point values, distance],
        total point value count
    ],
    [array of unknown area points coordinates]
]
```

***

## ```select_values_in_range()```

```python
pyinterpolate.transform.select_values_in_range(
    data,
    lag,
    step_size)
```

Function selects set of values which are greater than (```lag - step size / 2```) and smaller or equal than (```lag + step size / 2```).

INPUT:

- **data**: (_numpy array_) distances,
- **lag**: (_float_) lag within areas are included,
- **step_size**: (_float_) step between lags.


OUTPUT:

- ```numpy array``` mask with distances within specified radius.

***

## ```set_areal_weights()```

```python
pyinterpolate.transform.set_areal_weights(
    areal_data,
    areal_points)
```

Function prepares array for _weighted semivariance_ calculation.

INPUT:

- **areal_data**: (_numpy array_) of areas in the form:

```python
[area_id, areal_polygon, centroid coordinate x, centroid coordinate y, value]
```

- **areal_points**: (_numpy array_) of points within areas in the form:

```python
[area_id, [point_position_x, point_position_y, value]]
````


OUTPUT:

- ```numpy array``` of weighted points.